In [9]:
# Define the probability matrix
probabilities = {
    'C': {'C': 0.35, 'F': 0.85, 'T': 0.25,'R': 0.25, 'W': 0.10},
    'F': {'C': 0.15, 'F': 0.60, 'T': 0.50,'R': 0.65, 'W': 0.80},
    'T': {'C': 0.75, 'F': 0.36, 'T': 0.53,'R': 0.45, 'W': 0.40},
    'R': {'C': 0.60, 'F': 0.30, 'T': 0.50,'R': 0.40, 'W': 0.40},
    'W': {'C': 0.90, 'F': 0.20, 'T': 0.65,'R': 0.60, 'W': 0.60}
}

# All possible units
units= ['C', 'F', 'T', 'R', 'W']

target_branch=0


In [10]:
import pandas as pd
import numpy as np
#probs=pd.read_csv('probs.csv').drop('Unnamed: 0',axis=1).replace(np.NaN,50)

# df = pd.DataFrame(probabilities).reset_index()
# df.rename(columns={'index': 'class'}, inplace=True)
# df

In [11]:
import itertools

# Function to calculate the probability of winning at least 3 out of 5 games
def probability_of_winning_at_least_3(matchups):
    total_prob = 0
    for i in range(3, 6):
        for combo in itertools.combinations(matchups, i):
            remaining_elements=list(set(matchups)-set(combo))
            prob = 1
            for (a, b) in combo:
                p= probabilities[a][b]
                q=1-p
                prob *= p*p*p + 3*p*p*q
            for (a, b) in remaining_elements:
                p= probabilities[a][b]
                q=1-p
                prob *=(1-(p*p*p + 3*p*p*q))
            total_prob += prob 
    return total_prob 

def minimax(turn_order, game_index, a_units, b_units, current_matchups, memo,target_branch):

    # Generate a key for the memoization dictionary
    memo_key = (tuple(turn_order), game_index, tuple(a_units), tuple(b_units), tuple(current_matchups))
    
    if memo_key in memo:
        return memo[memo_key]

    # Base case: if all games are assigned
    if game_index == len(turn_order):
        prob = probability_of_winning_at_least_3(current_matchups)
        memo[memo_key] = (prob,current_matchups)
        #print(f'{current_matchups}, {prob}')
        return prob, current_matchups


    # Determine the current player's turn
    current_player = turn_order[game_index]
    is_turn_a = current_player == 'A'
    
    # Initialize the best value
    if is_turn_a:
        best_value = float('-inf')
    else:
        best_value = float('inf')

    best_option = None

    # Generate all possible choices for the current player
    if is_turn_a:
        choices = a_units
    else:
        choices = b_units
    
    for choice in choices:
        if is_turn_a:
            new_a_units = [unit for unit in a_units if unit != choice]
            new_b_units = b_units
        else:
            new_a_units = a_units
            new_b_units = [unit for unit in b_units if unit != choice]
        
       # Update matchups
        new_matchups = current_matchups.copy()
        if is_turn_a:
            # Player A's turn
            if game_index % 2 == 0:  # A's choices are in games 1, 4, 5, 7, 8 (0-based index)
                new_matchups.append((choice, None))  # A's choice is made, placeholder for B
            else:
                last_choice = new_matchups[-1][1]
                new_matchups[-1] = (choice, last_choice)  # Complete the pair
        else:
            # Player B's turn
            if game_index % 2 == 1:
                last_choice = new_matchups[-1][0]
                new_matchups[-1] = (last_choice, choice)  # Complete the pair
            else:
                new_matchups.append((None, choice))  # B's choice is made, placeholder for A

        
        # Continue to the next index
        #print(f"player: {current_player},game_index: {game_index}, {new_matchups},bv  {best_value}")
        value,option = minimax(turn_order, game_index + 1, new_a_units, new_b_units, new_matchups, memo,target_branch)
        #print(f'{new_matchups}, value {value}, best_value {best_value}')
        

        if is_turn_a:
            if value >= best_value:
                best_option=option
                if game_index in target_branch:
                    print(f'{turn_order[game_index]} {game_index} {option} choosen {value} > {best_value}')
            else:
                if game_index in target_branch: 
                    print(f'{turn_order[game_index]} {game_index} {option} not choosen {value} < {best_value}')
                    #print(f'{turn_order[game_index]} {game_index} {best_option} best option: {turn_order[game_index]} {value} < {best_value}: NO')
            best_value = max(best_value, value)
        else:
            if value <= best_value:
                best_option=option
                if game_index in target_branch:
                    print(f'{turn_order[game_index]} {game_index} {option} choosen {value} < {best_value}')
            else:
                if game_index in target_branch:
                    print(f'{turn_order[game_index]} {game_index} {option} not choosen {value} > {best_value}')
                    #print(f'{turn_order[game_index]} {game_index} {best_option} best option: {turn_order[game_index]} {value} > {best_value}: NO')
            best_value = min(best_value, value)
    
    memo[memo_key] = (best_value,best_option)

    return best_value, best_option

# Initialize memoization dictionary
memo = {}

# Define the turn order
turn_order = ['A', 'B', 'B', 'A', 'A', 'B', 'B', 'A', 'A', 'B']

# Starting state: Player A's turn, all units available, no matchups yet, game index 0
initial_matchups = []

# Compute the optimal probability
target_branch=[9]
optimal_probability,best_option = minimax(turn_order, 0, units, units, initial_matchups, memo,target_branch)
print(f'Optimal probability of winning at least 3 games: {optimal_probability}')
print(f"Best sequence of matchups: {best_option}")

B 9 [('C', 'C'), ('F', 'F'), ('T', 'T'), ('R', 'R'), ('W', 'W')] choosen 0.4911505724262422 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'T'), ('W', 'R'), ('R', 'W')] choosen 0.49115057242624216 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'T'), ('R', 'W'), ('W', 'R')] choosen 0.4911505724262422 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'T'), ('W', 'W'), ('R', 'R')] choosen 0.49115057242624216 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'R'), ('R', 'T'), ('W', 'W')] choosen 0.5019194362499999 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'R'), ('W', 'T'), ('R', 'W')] choosen 0.468901857750514 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'R'), ('R', 'W'), ('W', 'T')] choosen 0.468901857750514 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'R'), ('W', 'W'), ('R', 'T')] choosen 0.5019194362499999 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'W'), ('R', 'T'), ('W', 'R')] choosen 0.4728868719999999 < inf
B 9 [('C', 'C'), ('F', 'F'), ('T', 'W'), ('W', 'T'), ('R', 'R')] choosen 0.43842775848931204 < inf
B 9 [('C', 'C'), ('

In [12]:
mdf=pd.DataFrame(memo)
mdf=mdf.transpose()
mdf.reset_index(drop=True,inplace=True)
mdf=mdf.iloc[mdf.astype(str).drop_duplicates().index]
mdf.sort_values(by=0, ascending=False,inplace=True)